# Sidemantic + DuckDB: SQL Model Definitions

This notebook shows how to define a semantic layer with Sidemantic's SQL model format and query it using DuckDB.

Prereqs (Colab):

- `pip install sidemantic duckdb polars pyarrow`


In [ ]:
# Colab setup
!pip -q install sidemantic duckdb polars pyarrow

In [ ]:
import tempfile
from pathlib import Path

import duckdb

from sidemantic import SemanticLayer

workdir = Path(tempfile.mkdtemp(prefix="sidemantic_demo_"))
db_path = workdir / "demo.duckdb"

con = duckdb.connect(str(db_path))
con.execute(
    """
    create table orders as
    select * from (values
        (1, '2025-01-01', 'completed', 120.0, 'us', 101),
        (2, '2025-01-02', 'completed', 80.0, 'us', 102),
        (3, '2025-01-03', 'returned', 60.0, 'eu', 101),
        (4, '2025-01-04', 'completed', 200.0, 'eu', 103),
        (5, '2025-01-05', 'pending', 50.0, 'us', 104)
    ) as t(order_id, order_date, status, amount, region, customer_id);
    """
)

df = con.execute("select * from orders").pl()
con.close()
df

In [ ]:
sql_model = """
MODEL (
    name orders,
    table orders,
    primary_key order_id
);

DIMENSION (
    name order_date,
    type time,
    sql order_date,
    granularity day
);

DIMENSION (
    name status,
    type categorical,
    sql status
);

DIMENSION (
    name region,
    type categorical,
    sql region
);

METRIC (
    name revenue,
    agg sum,
    sql amount
);

METRIC (
    name order_count,
    agg count
);

METRIC (
    name avg_order_value,
    agg avg,
    sql amount
);

""".strip()

sql_path = workdir / "orders.sidemantic.sql"
sql_path.write_text(sql_model)

sql_model

In [ ]:
from sidemantic.adapters.sidemantic import SidemanticAdapter

adapter = SidemanticAdapter()
graph = adapter.parse(sql_path)

layer = SemanticLayer(connection=f"duckdb:///{db_path}")
layer.graph = graph

result = layer.query(
    metrics=["orders.revenue", "orders.order_count", "orders.avg_order_value"],
    dimensions=["orders.status"],
)

result.pl()

In [ ]:
layer.sql(
    """
    select
        status,
        revenue,
        order_count,
        avg_order_value
    from orders
    where status != 'pending'
    order by revenue desc
    """
).pl()

In [ ]:
print(
    layer.compile(
        metrics=["orders.revenue", "orders.order_count"],
        dimensions=["orders.region"],
    )
)